# Backtesting assets and a benchmark both b&h and RSI strategy

### In this test we are using vectorbt and pandas_ta (pandas_ta is optional, but is useful

In [ ]:
# Define functions to connect to Google and change directories
def connectDrive():
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

def changeDirectory(path):
    import os
    original_path = os.getcwd()
    os.chdir(path)
    new_path = os.getcwd()
    print("Original path: ",original_path)
    print("New path: ",new_path)

# Connect to Google Drive
connectDrive()

# Change path
changeDirectory("/content/drive/My Drive/github/find_p/")

In [ ]:
!pip install vectorbt
!pip install pandas_ta
!pip install yfinance
!pip install plotly
!pip install numba
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca

In [1]:
import asyncio
import itertools
from datetime import datetime

from IPython import display

import numpy as np
import pandas as pd
import vectorbt as vbt

from pdta_vt_utils import *

import plotly.graph_objects as go
%matplotlib inline

# Configuration of the backtesting platform

### Important step that need check before any experiment

In [2]:
cheight, cwidth = 500, 1000 # Adjust as needed for Chart Height and Width
vbt.settings.set_theme("dark") # Options: "light" (Default), "dark" (my fav), "seaborn"

# Must be set
vbt.settings.portfolio["freq"] = "1D" # Daily

# Predefine vectorbt Portfolio settings
vbt.settings.portfolio["init_cash"] = 1000
vbt.settings.portfolio["fees"] = 0.0025 # 0.25%
vbt.settings.portfolio["slippage"] = 0.0025 # 0.25%
# vbt.settings.portfolio["size"] = 100
# vbt.settings.portfolio["accumulate"] = False
vbt.settings.portfolio["allow_partial"] = False

pf_settings = pd.DataFrame(vbt.settings.portfolio.items(), columns=["Option", "Value"])
pf_settings.set_index("Option", inplace=True)

print(f"Portfolio Settings [Initial]")
pf_settings


Portfolio Settings [Initial]


,Value
Option,
call_seq,default
init_cash,1000
size,inf
size_type,amount
fees,0.0025
fixed_fees,0.0
slippage,0.0025
reject_prob,0.0
min_size,0.0


# Setting of the assets to test

### It is not dynamic and do not offer any advantage but is a way to have the data collected

In [3]:
benchmark_tickers = ["SPY", "QQQ"]
asset_tickers = ["AAPL", "TSLA", "TWTR", "SPXL", "^GSPC", "SPY"]
all_tickers = benchmark_tickers + asset_tickers

print("Tickers by index #")
print("="*100)
print(f"Benchmarks: {', '.join([f'{k}: {v}' for k,v in enumerate(benchmark_tickers)])}")
print(f"    Assets: {', '.join([f'{k}: {v}' for k,v in enumerate(asset_tickers)])}")
print(f"       All: {', '.join([f'{k}: {v}' for k,v in enumerate(all_tickers)])}")
print("="*100)
benchmarks = dl(benchmark_tickers, lc_cols=True)
assets = dl(asset_tickers, lc_cols=True)

Tickers by index #
Benchmarks: 0: SPY, 1: QQQ
    Assets: 0: AAPL, 1: TSLA, 2: TWTR, 3: SPXL, 4: ^GSPC, 5: SPY
       All: 0: SPY, 1: QQQ, 2: AAPL, 3: TSLA, 4: TWTR, 5: SPXL, 6: ^GSPC, 7: SPY


In [4]:
########### You need to set this #############################
# Numero del indice que se quiere ver tanto para exploracion com para benchmark e.j. spy i=5
benchmark_name = benchmark_tickers[0] # Change index for different benchmark
asset_name = asset_tickers[3] # Change index for different symbol
#################################################################

print("="*100)
print(f"Selected Benchmark | Asset: {benchmark_name} | {asset_name}")
start_date = datetime(2010, 1, 1) # Adjust as needed
start_date = pd.to_datetime(start_date).tz_localize('America/New_York')
end_date = datetime(2015, 1, 1)   # Adjust as needed
end_date = pd.to_datetime(end_date).tz_localize('America/New_York')
print("Available Data:")
print("="*100)
print(f"Benchmarks: {', '.join(benchmarks.keys())}")
print(f"Assets: {', '.join(assets.keys())}")
print("="*100)

benchmarkdf = benchmarks[benchmark_name]
assetdf     = assets[asset_name]

# Set True if you want to constrain Data between start_date & end_date
common_range = True
crs = ''
if common_range:
    crs = f" from {start_date} to {end_date}"
    benchmarkdf = dtmask(benchmarkdf, start_date, end_date)
    assetdf = dtmask(assetdf, start_date, end_date)

# Update DataFrame names
benchmarkdf.name = benchmark_name
assetdf.name = asset_name
print(f"Analysis of: {benchmarkdf.name} and {assetdf.name}{crs}")

Selected Benchmark | Asset: SPY | SPXL
[i] Downloading: SPY, QQQ
[+] SPY(7565, 8) Monday February 13, 2023, NYSE: 14:42:31
[+] QQQ(6023, 8) Monday February 13, 2023, NYSE: 14:42:31
[*] Download Complete

[i] Downloading: AAPL, TSLA, TWTR, SPXL, ^GSPC, SPY
[+] AAPL(10632, 7) Monday February 13, 2023, NYSE: 14:42:32
[+] TSLA(3179, 7) Monday February 13, 2023, NYSE: 14:42:32
[+] TWTR(2259, 7) Monday February 13, 2023, NYSE: 14:42:32
[+] SPXL(3592, 8) Monday February 13, 2023, NYSE: 14:42:33
[+] ^GSPC(23894, 7) Monday February 13, 2023, NYSE: 14:42:33
[+] SPY(7565, 8) Monday February 13, 2023, NYSE: 14:42:34
[*] Download Complete



In [7]:
benchmarkdf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2010-01-04 00:00:00-05:00,87.704935,88.501044,87.033704,88.454216,118944600,0.0,0.0,0.0
2010-01-05 00:00:00-05:00,88.399588,88.727397,88.079580,88.688370,111579900,0.0,0.0,0.0
2010-01-06 00:00:00-05:00,88.602481,88.969318,88.532239,88.750778,116074400,0.0,0.0,0.0
2010-01-07 00:00:00-05:00,88.586872,89.234689,88.337112,89.125420,131091100,0.0,0.0,0.0
2010-01-08 00:00:00-05:00,88.891294,89.461063,88.711782,89.422035,126402800,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2014-12-24 00:00:00-05:00,179.701043,179.977473,179.441881,179.485077,42963400,0.0,0.0,0.0
2014-12-26 00:00:00-05:00,179.951567,180.418061,179.899737,180.063873,57326700,0.0,0.0,0.0
2014-12-29 00:00:00-05:00,179.873808,180.521706,179.804698,180.305740,79643900,0.0,0.0,0.0


In [8]:
assetdf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2010-01-04 00:00:00-05:00,4.100748,4.207281,4.099987,4.200432,28238400,0.0,0.0,0.0
2010-01-05 00:00:00-05:00,4.195867,4.242285,4.147167,4.240002,33206400,0.0,0.0,0.0
2010-01-06 00:00:00-05:00,4.227827,4.278810,4.217935,4.251416,44194800,0.0,0.0,0.0
2010-01-07 00:00:00-05:00,4.229349,4.315336,4.188258,4.301639,43773600,0.0,0.0,0.0
2010-01-08 00:00:00-05:00,4.264352,4.352622,4.240001,4.347294,39685200,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2014-12-24 00:00:00-05:00,21.372383,21.452077,21.266905,21.290344,1291600,0.0,0.0,0.0
2014-12-26 00:00:00-05:00,21.445040,21.606775,21.421600,21.480200,2223200,0.0,0.0,0.0
2014-12-29 00:00:00-05:00,21.400508,21.634906,21.391132,21.559898,2677600,0.0,0.0,0.0


In [9]:

def trends(df: pd.DataFrame, below: int = 30, above: int = 70):
    rsi = vbt.RSI.run(df.Close, window = 14, short_name="rsi")
    df['TS_Entries'] = rsi.rsi_crossed_below(below)
    df['TS_Exits'] = rsi.rsi_crossed_above(above)
    df['TS_Trend'] = rsi.rsi_crossed_below(below) | rsi.rsi_crossed_above(above)
    return df, rsi

# def trends(df: pd.DataFrame, mamode: str = "sma", fast: int = 50, slow: int = 200):
#     rsi = df.ta.rsi()
#     return ta.xsignals(rsi, 40, 85, above=True, asbool=True)

In [10]:
####### Configuration for the strategy ###########
# You can chenge this, remember for example you can set the default values
# We are adding stop loss as a parameter here
###################################################
trend_kwargs = {"below": 24, "above": 89}
sl_stop = 0.0451
#################################################

benchmark_trends, _ = trends(benchmarkdf, **trend_kwargs)
# benchmark_trends.TS_Trend.copy().astype(int).plot(figsize=(16, 1), kind="area", color=["green"], alpha=0.45, title=f"{benchmarkdf.name} Trends", grid=True)

asset_trends, _ = trends(assetdf, **trend_kwargs)
# asset_trends.TS_Trend.copy().astype(int).plot(figsize=(16, 1), kind="area", color=["green"], alpha=0.45, title=f"{assetdf.name} Trends", grid=True)

In [11]:
# # trade_offset = 0 for Live Signals (close is last price)
# # trade_offset = 1 for Backtesting
LIVE = 0

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,TS_Entries,TS_Exits,TS_Trend
Date,,,,,,,,,,,
2010-01-04 00:00:00-05:00,4.100748,4.207281,4.099987,4.200432,28238400,0.0,0.0,0.0,False,False,False
2010-01-05 00:00:00-05:00,4.195867,4.242285,4.147167,4.240002,33206400,0.0,0.0,0.0,False,False,False
2010-01-06 00:00:00-05:00,4.227827,4.278810,4.217935,4.251416,44194800,0.0,0.0,0.0,False,False,False
2010-01-07 00:00:00-05:00,4.229349,4.315336,4.188258,4.301639,43773600,0.0,0.0,0.0,False,False,False
2010-01-08 00:00:00-05:00,4.264352,4.352622,4.240001,4.347294,39685200,0.0,0.0,0.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2014-12-24 00:00:00-05:00,21.372383,21.452077,21.266905,21.290344,1291600,0.0,0.0,0.0,False,False,False
2014-12-26 00:00:00-05:00,21.445040,21.606775,21.421600,21.480200,2223200,0.0,0.0,0.0,False,False,False
2014-12-29 00:00:00-05:00,21.400508,21.634906,21.391132,21.559898,2677600,0.0,0.0,0.0,False,False,False


# Backtest the strategy by signals

### Signal Portfolios with their Last 'k' Trades and Performance Statistics

In [12]:
# Benchmark Portfolio from Trade Signals
benchmarkpf_signals = vbt.Portfolio.from_signals(benchmarkdf.Close,
                                                 entries=benchmark_trends.TS_Entries,
                                                 exits=benchmark_trends.TS_Exits,
                                                 sl_stop=sl_stop)
trade_table(benchmarkpf_signals, k=5)
combine_stats(benchmarkpf_signals, benchmarkdf.name, "Long Strategy", LIVE)

In [13]:
# Asset Portfolio from Trade Signals
assetpf_signals = vbt.Portfolio.from_signals(assetdf.Close,
                                             entries=asset_trends.TS_Entries,
                                             exits=asset_trends.TS_Exits,
                                             sl_stop=sl_stop)
trade_table(assetpf_signals, k=5)
combine_stats(assetpf_signals, assetdf.name, "Long Strategy", LIVE)


Last 5 of 7 Trades
   status  direction       size  entry_price  exit_price    return  \
2       1          0  12.113762    90.421627  116.045303  0.277672   
3       1          0  12.256219   114.124428  124.815818  0.088448   
4       1          0  11.001908   138.352388  143.108236  0.029289   
5       1          0   9.566335   163.762924  170.810733  0.037929   
6       0          0   9.538545   170.453972  177.558670  0.039181   

          pnl  entry_fees  exit_fees  
2  304.146395    2.738365   3.514363  
3  123.714755    3.496835   3.824425  
4   44.581893    3.805351   3.936159  
5   59.420094    3.916527   4.085082  
6   63.703771    4.064707   0.000000  



NameError: name 'LIVE' is not defined

# Signal Plots

### Benchmark

In [ ]:
assetpf_signals.plot().show_png()

# Just Continue runing if you are satisfied with the backtest, if not run first the optimization and find the good parameters for the indicator.

In [ ]:
benchmarkpf_signals.trades.plot(title=f"{benchmarkdf.name} | Trades", height=cheight, width=cwidth).show_png()

In [ ]:
benchmarkpf_signals.value().vbt.plot(title=f"{benchmarkdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
benchmarkpf_signals.drawdown().vbt.plot(title=f"{benchmarkdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
benchmarkpf_signals.trades.plot_pnl(title=f"{benchmarkdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()

In [ ]:
benchmarkpf_signals.returns().vbt.plot(title=f"{benchmarkdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
benchmarkpf_signals.cash().vbt.plot(title=f"{benchmarkdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
total_assetfees = benchmarkpf_signals.trades.records_readable["Entry Fees"] + benchmarkpf_signals.trades.records_readable["Exit Fees"]
total_assetfees.vbt.plot(title=f"{benchmarkdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

### Asset

In [ ]:
assetpf_signals.trades.plot(title=f"{assetdf.name} | Trades", height=cheight, width=cwidth).show_png()

In [ ]:
assetpf_signals.value().vbt.plot(title=f"{assetdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.drawdown().vbt.plot(title=f"{assetdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.trades.plot_pnl(title=f"{assetdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.returns().vbt.plot(title=f"{assetdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
assetpf_signals.cash().vbt.plot(title=f"{assetdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
total_assetfees = assetpf_signals.trades.records_readable["Entry Fees"] + assetpf_signals.trades.records_readable["Exit Fees"]
total_assetfees.vbt.plot(title=f"{assetdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

# Uncomment and run the follwing just if you want a benchmark with buy and hold in details, remember the first plot containg the buy and hold of the Asset

In [ ]:
# Benchmark Buy and Hold (BnH) Strategy
# benchmarkpf_bnh = vbt.Portfolio.from_holding(benchmarkdf.Close)
# print(trade_table(benchmarkpf_bnh))
# combine_stats(benchmarkpf_bnh, benchmarkdf.name, "Buy and Hold", LIVE)
#### Benchmark Buy and Hold Plots #######################
# vbt.settings.set_theme("seaborn")
# benchmarkpf_bnh.trades.plot(title=f"{benchmarkdf.name} | Trades", height=cheight, width=cwidth).show_png()
# benchmarkpf_bnh.value().vbt.plot(title=f"{benchmarkdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.drawdown().vbt.plot(title=f"{benchmarkdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.trades.plot_pnl(title=f"{benchmarkdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.returns().vbt.plot(title=f"{benchmarkdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# benchmarkpf_bnh.cash().vbt.plot(title=f"{benchmarkdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# total_assetfees = benchmarkpf_bnh.trades.records_readable["Entry Fees"] + benchmarkpf_bnh.trades.records_readable["Exit Fees"]
# total_assetfees.vbt.plot(title=f"{benchmarkdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()

In [ ]:
# Asset Buy and Hold (BnH) Strategy
# assetpf_bnh = vbt.Portfolio.from_holding(assetdf.Close)
# print(trade_table(assetpf_bnh))
# combine_stats(assetpf_bnh, assetdf.name, "Buy and Hold", LIVE)
### Asset Buy and Hold ############################
# vbt.settings.set_theme("seaborn")
# assetpf_bnh.trades.plot(title=f"{assetdf.name} | Trades", height=cheight, width=cwidth).show_png()
# assetpf_bnh.value().vbt.plot(title=f"{assetdf.name} | Equity Curve", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.drawdown().vbt.plot(title=f"{assetdf.name} | Drawdown", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.trades.plot_pnl(title=f"{assetdf.name} | PnL", height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.returns().vbt.plot(title=f"{assetdf.name} | Active Returns", trace_kwargs=dict(name="%"), height=cheight // 2, width=cwidth).show_png()
# assetpf_bnh.cash().vbt.plot(title=f"{assetdf.name} | Cash", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()
# total_assetfees = assetpf_bnh.trades.records_readable["Entry Fees"] + assetpf_bnh.trades.records_readable["Exit Fees"]
# total_assetfees.vbt.plot(title=f"{assetdf.name} | Total Fees", trace_kwargs=dict(name=u"\u00A4"), height=cheight // 2, width=cwidth).show_png()